# Introducing Snorkel

In this notebook we will use Snorkel to enrich our data such that tags with between 500-2,000 examples will be labeled using weak supervision to produce labels for enough examples to allow us to train an accurate full model that includes these new labels.

More information about Snorkel can be found at [Snorkel.org](https://www.snorkel.org/) :) For a basic introduction to Snorkel, see the [Spam Tutorial](http://syndrome:8888/notebooks/snorkel-tutorials/spam/01_spam_tutorial.ipynb). For an introduction to Multi-Task Learning (MTL), see [Multi-Task Tutorial](http://syndrome:8888/notebooks/snorkel-tutorials/multitask/multitask_tutorial.ipynb).

In [30]:
# Snorkel Introduction

from collections import OrderedDict 
import gc
import os
import sys

import numpy as np
import pandas as pd
import pyarrow
import random
import snorkel
import spacy
import tensorflow as tf
from spacy.pipeline import merge_entities

# Add parent directory to path
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)

# Make reproducible
random.seed(1337)

# Turn off TensorFlow logging messages
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# For reproducibility
os.environ["PYTHONHASHSEED"] = "1337"

# Show wide columns
pd.set_option('display.max_colwidth', 300)

In [37]:
TAG_LIMIT = 50

PATHS = {
    'questions': {
        'local': '../data/stackoverflow/Questions.Tags.{}.parquet',
        'local_single': '../data/stackoverflow/Questions.Tags.{}.parquet/part-00029-1ad544ea-abd4-4960-aa2c-7e0eb12cdb8e-c000.snappy.parquet',
        's3': 's3://stackoverflow-events/08-05-2019/Questions.Tags.{}.parquet',
    },
    'questions_parts_json': {
        's3': 's3://stackoverflow-events/08-05-2019/Entity.Candidates.{}.jsonl',
    },
    'questions_parts_parquet': {
        's3': 's3://stackoverflow-events/08-05-2019/Entity.Candidates.{}.parquet',
    },
    'entity_candidates': {
        'local': '../data/stackoverflow/Entity.Candidates.{}.parquet',
        'local_single': '../data/stackoverflow/Entity.Candidates.{}.parquet/part-00000*.parquet',
        's3': 's3://stackoverflow-events/08-05-2019/Entity.Candidates.{}.parquet',
    },
    'gold_labels': {
        's3': 's3://stackoverflow-events/text_extractions.one_file.df_out.gold.labeled.final.csv',
        'local': '../data/text_extractions.one_file.df_out.gold.labeled.final.csv',
        'local_single': '../data/text_extractions.one_file.df_out.gold.labeled.final.csv',
    }
}

# Define a set of paths for each step for local and S3
PATH_SET = 'local_single' # 's3'

path = PATHS['questions'][PATH_SET].format(TAG_LIMIT)

## Loading our Examples for Augmentation in Pandas

In [35]:
df = pd.read_parquet(
    path, 
    engine='pyarrow',
)
df.head(1)

,_PostId,_AcceptedAnswerId,_Body,_Code,_Tags,_Label,_AnswerCount,_CommentCount,_FavoriteCount,_OwnerUserId,...,_AccountId,_UserId,_UserDisplayName,_UserDownVotes,_UserLocation,_ProfileImageUrl,_UserReputation,_UserUpVotes,_UserViews,_UserWebsiteUrl
0,264,NaN,BerkeleyDB Concurrency \nWhat's the optimal level of concurrency that the C++ implementation of BerkeleyDB can reasonably support?\nHow many threads can I have hammering away at the DB before throughput starts to suffer because of resource contention?\n\nI've read the manual and know how to set ...,,"[c++, berkeley-db]",0,5,0,1.0,104,...,86,104,Ted Dziuba,4,California,None,1600,9,2325,http://www.teddziuba.com/


## Loading our Examples for Augmentation in PySpark

In [ ]:
from pyspark import SparkContext
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql import SparkSession
from pyspark.sql import Row

spark = SparkSession \
    .builder \
    .appName("Programming Language Extraction Example") \
    .getOrCreate()
sc = spark.sparkContext

question_df = spark.read.parquet(path)
# question_df.limit(3).toPandas()
question_df.show()

## Enable spaCy GPU Support

That is, if you have a GPU and are using Pandas. `multiprocessing.Pool` Pandas and PySpark can't use a GPU yet.

In [ ]:
# spacy.prefer_gpu()

## Create a spaCy [`Language`](https://spacy.io/api/language) Model

In [18]:
# Download the spaCy english model
spacy.cli.download('en_core_web_lg')
nlp = spacy.load("en_core_web_lg", disable=["vectors"])

# Merge multi-token entities together
nlp.add_pipe(merge_entities)

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [19]:
nlp.pipeline

[('tagger', <spacy.pipeline.pipes.Tagger at 0x7f0da9e033d0>),
 ('parser', <spacy.pipeline.pipes.DependencyParser at 0x7f0da9eb6fa0>),
 ('ner', <spacy.pipeline.pipes.EntityRecognizer at 0x7f0da9eb6ec0>),
 ('merge_entities', <function spacy.pipeline.functions.merge_entities(doc)>)]

## Exploring spaCy

Below we use print statements and the visualization tool  `spacy.displacy` to render parsed objects for an example document. First we iterate the spaCy  [`Token`s](https://spacy.io/api/token) that make up the [`Doc`](https://spacy.io/api/doc) and print the text and the string defining their part-of-speech. 

We’ll be using these parts of speech to write Labeling Functions using spaCy pattern matching. It will be very useful to know the part of speech we’re looking for in our entities - almost exclusively proper nouns - `PROPN` - and quite often of the pattern `VERB-ADP-PROPN`, which we’ll see below.

Next we use `spacy.displacy` to visualize the parse tree of dependencies between words as well as the entities detected in the sentence. This gives a rough idea of the structure that a spaCy `Doc` has for us to use. It also creates dense vectors based on embeddings for the words and the entire document, which we can use to create LFs as well.

In [20]:
from spacy import displacy

s = 'The program to do payroll was written in C++ and Perl.'
d = nlp(s)
tups = []
for t in d:
    tups.append((t.text, t.pos_))

# Print words/parts-of-speech
print([x for x in tups])

# Render image diagrams
displacy.render(d, style='dep', options={'compact': True, 'collapse_punct': True, 'distance': 90}, )
displacy.render(d, style='ent')

[('The', 'DET'), ('program', 'NOUN'), ('to', 'PART'), ('do', 'AUX'), ('payroll', 'NOUN'), ('was', 'AUX'), ('written', 'VERB'), ('in', 'ADP'), ('C++', 'PROPN'), ('and', 'CCONJ'), ('Perl', 'PROPN'), ('.', 'PUNCT')]


## In Pandas, produce records with their left/right tokens for all entities in all documents.

In [ ]:
#
# Pandas - single processor
#
window = 4
candidates = []
for index, row in df.iterrows():
    doc = nlp(row['_Body'])
    re_doc_1 = nlp(row['body'])
    re_doc_2 = nlp(row['body'])
    
    for ent in doc.ents:
        rec = {}
        rec['body'] = doc.text

        rec['entity_text'] = ent.text
        rec['entity_start'] = ent.start
        rec['entity_end'] = ent.end
        rec['ent_type'] = ent.label_

        # Retokenize the tokens to the left of the entity into a single string
        # using a new Doc without messing up the original Doc
        left_token_start = max(0, ent.start - 1 - window)
        left_token_end = ent.start
        rec['left_tokens_text'] = [x.text for x in doc[left_token_start : left_token_end]]

        left_merged_token = re_doc_1[left_token_start: left_token_end].merge()
        rec['left_text'] = left_merged_token.text if left_merged_token else ''
        del left_token_start
        del left_token_end
        del left_merged_token

        # Retokenize the tokens to the left of the entity into a single string
        # using a new Doc without messing up the original Doc
        right_token_start = min(ent.end, len(doc) - 1)
        right_token_end = min(ent.end + window, len(doc) - 1)
        rec['right_tokens_text'] = [x.text for x in doc[right_token_start : right_token_end]]

        right_merged_token = re_doc_2[right_token_start: right_token_end].merge()
        rec['right_text'] = right_merged_token.text if right_merged_token else ''
        del right_token_start
        del right_token_end
        del right_merged_token

        rec['wikidata_id'] = ent.kb_id
        
        rec['original_index'] = index
        rec['label'] = 0

        candidates.append(rec)
        del ent
    
    del doc
    del re_doc_1
    del re_doc_2
    gc.collect()

df_out = pd.DataFrame(candidates)
df_out = df_out.reindex().sort_index()

df_out.head()

## Use `multiprocessing.Pool` to Speed up Pandas

In [27]:
def process_split(df: pd.DataFrame, window: int=5):

    candidates = []
    for index, row in df.iterrows():
        
        # Need 3 docs since retokenizing the left/right tokens shortens the Doc, 
        # messing up our start/stop indexes and Docs can't be copied.
        doc = nlp(row['_Body'])
        re_doc_1 = nlp(row['_Body'])
        re_doc_2 = nlp(row['_Body'])

        for ent in doc.ents:
            rec = {}
            rec['body'] = doc.text

            rec['entity_text'] = ent.text
            rec['entity_start'] = ent.start
            rec['entity_end'] = ent.end
            rec['ent_type'] = ent.label_

            # Retokenize the tokens to the left of the entity into a single string
            # using a new Doc without messing up the original Doc
            left_token_start = max(0, ent.start - 1 - window)
            left_token_end = ent.start
            rec['left_tokens_text'] = [x.text for x in doc[left_token_start : left_token_end]]
            
            left_merged_token = re_doc_1[left_token_start: left_token_end].merge()
            rec['left_text'] = left_merged_token.text if left_merged_token else ''
            del left_token_start
            del left_token_end
            del left_merged_token
            
            # Retokenize the tokens to the left of the entity into a single string
            # using a new Doc without messing up the original Doc
            right_token_start = min(ent.end, len(doc) - 1)
            right_token_end = min(ent.end + window, len(doc) - 1)
            rec['right_tokens_text'] = [x.text for x in doc[right_token_start : right_token_end]]
            
            right_merged_token = re_doc_2[right_token_start: right_token_end].merge()
            rec['right_text'] = right_merged_token.text if right_merged_token else ''
            del right_token_start
            del right_token_end
            del right_merged_token

            rec['wikidata_id'] = ent.kb_id

            rec['original_index'] = index
            rec['label'] = 0

            candidates.append(rec)
            del ent
            
        del doc
        del re_doc_1
        del re_doc_2
        gc.collect()

    df_out = pd.DataFrame(candidates)
    df_out = df_out.reindex().sort_index()
    
    return df_out

In [29]:
from multiprocessing import cpu_count, Pool


def restore_spacy(df, n_cores=cpu_count):
    
    n_cores = cpu_count() if callable(cpu_count) else cpu_count
    
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    
    df_out = pd.concat(
        pool.map(
            process_split,
            df_split
        )
    )
    df_out = df_out.reindex().sort_index()
    
    pool.close()
    pool.join()
    
    return df_out

df_out = restore_spacy(df)

df_out.head()

In [ ]:
entity_path = PATHS['entity_candidates'][PATH_SET].format(TAG_LIMIT)

df_out.to_parquet(
    entity_path,
    engine='pyarrow'
)

## In PySpark, produce records with their left/right tokens for all entities in all documents.

In [ ]:
from typing import Any

def prepare_docs(rows: Any, window: int=5):
    
    nlp = spacy.load("en_core_web_lg", disable=["vectors"])
    nlp.add_pipe(merge_entities)
    
    recs = []
    
    for row in rows:
        
        # Need 3 docs since retokenizing the left/right tokens shortens the Doc, 
        # messing up our start/stop indexes
        doc = nlp(row._Body)
        re_doc_1 = nlp(row._Body)
        re_doc_2 = nlp(row._Body)
        
        for ent in doc.ents:
            rec = {}
            rec['body'] = doc.text

            rec['entity_text'] = ent.text
            rec['entity_start'] = ent.start
            rec['entity_end'] = ent.end
            rec['ent_type'] = ent.label_
            
            # Retokenize the tokens to the left of the entity into a single string
            # using a new Doc without messing up the original Doc
            left_token_start = max(0, ent.start - 1 - window)
            left_token_end = ent.start
            rec['left_tokens_text'] = [x.text for x in doc[left_token_start : left_token_end]]

            left_merged_token = re_doc_1[left_token_start: left_token_end].merge()
            rec['left_text'] = left_merged_token.text if left_merged_token else ''
            del left_token_start
            del left_token_end
            del left_merged_token
            
            # Retokenize the tokens to the left of the entity into a single string
            # using a new Doc without messing up the original Doc
            right_token_start = min(ent.end, len(doc) - 1)
            right_token_end = min(ent.end + window, len(doc) - 1)
            rec['right_tokens_text'] = [x.text for x in doc[right_token_start : right_token_end]]
            
            right_merged_token = re_doc_2[right_token_start: right_token_end].merge()
            rec['right_text'] = right_merged_token.text if right_merged_token else ''
            del right_token_start
            del right_token_end
            del right_merged_token
            
            rec['wikidata_id'] = ent.kb_id
            rec['label'] = 0
            
            recs.append(rec)
            del ent
            
        del doc
        del re_doc_1
        del re_doc_2
        gc.collect()
    
    return recs

# Repartition the data to work with many mappers
question_partitioned_df = question_df.repartition(500, F.col('_PostId'))
entity_rdd = question_partitioned_df.rdd.mapPartitions(prepare_docs)

In [ ]:
# Write the data as JSON Lines
entity_json_path = PATHS['questions_parts_json'][PATH_SET].format(TAG_LIMIT)
entity_rdd.map(lambda x: json.dumps(x)).saveAsTextFile(entity_json_path)

# Load as JSON Lines into a DataFrame then store as Parquet (way faster than converting the RDD to DataFrame)
entity_df = spark.read.json(entity_json_path)
entity_parquet_path = PATHS['questions_parts_parquet'][PATH_SET].format(TAG_LIMIT)
entity_df.write.mode('overwrite').parquet(entity_parquet_path)
entity_df = spark.read.parquet(entity_parquet_path)

## Load the Gold Labeled data

In [51]:
import ast

gold_path = PATHS['gold_labels'][PATH_SET].format(TAG_LIMIT)
df_gold = pd.read_csv(gold_path)

# Drop the index column, we have an index set
df_gold = df_gold.drop(['Unnamed: 0'], axis=1)

df_gold['left_text'] = df_gold['left_text'].fillna('')
df_gold['right_text'] = df_gold['right_text'].fillna('')
df_gold['left_tokens_text'] = df_gold['left_tokens_text'].apply(lambda x: ast.literal_eval(x))
df_gold['right_tokens_text'] = df_gold['right_tokens_text'].apply(lambda x: ast.literal_eval(x))

df_gold.tail()

# Start the rest of the data after the point where the labeled data starts
# df_in = df_out.iloc[df_gold.index[-1] + 1:, :]
# df_in.head()

,body,entity_text,left_tokens_text,right_tokens_text,entity_start,ent_type,wikidata_id,entity_end,original_index,label,left_text,right_text
1285,What features are supported by Android's Google accounts authenticator? The API documentation for the method of Android's has the following to say about which features are supported by each authenticator:\n\nAccount features are authenticator-specific string tokens identifying\n boolean accou...,API,"['s, Google, accounts, authenticator, ?, The]","[documentation, for, the, , method]",12,ORG,0,13,305,0,'s Google accounts authenticator? The,documentation for the method
1286,What features are supported by Android's Google accounts authenticator? The API documentation for the method of Android's has the following to say about which features are supported by each authenticator:\n\nAccount features are authenticator-specific string tokens identifying\n boolean accou...,Android,"[documentation, for, the, , method, of]","['s, , has, the, following]",19,ORG,0,20,305,0,documentation for the method of,'s has the following
1287,What features are supported by Android's Google accounts authenticator? The API documentation for the method of Android's has the following to say about which features are supported by each authenticator:\n\nAccount features are authenticator-specific string tokens identifying\n boolean accou...,Google,"[are, used, to, tell, \n , whether]","[accounts, have, a, particular, service]",61,ORG,0,62,305,0,are used to tell\n whether,accounts have a particular service
1288,What features are supported by Android's Google accounts authenticator? The API documentation for the method of Android's has the following to say about which features are supported by each authenticator:\n\nAccount features are authenticator-specific string tokens identifying\n boolean accou...,Google,"[a, particular, service, (, such, as]","[\n , Calendar, or, Google Talk, )]",70,ORG,0,71,305,0,a particular service (such as,\n Calendar or Google Talk)
1289,What features are supported by Android's Google accounts authenticator? The API documentation for the method of Android's has the following to say about which features are supported by each authenticator:\n\nAccount features are authenticator-specific string tokens identifying\n boolean accou...,Google Talk,"[such, as, Google, \n , Calendar, or]","[), enabled, ., The, feature]",74,ORG,0,75,305,0,such as Google\n Calendar or,) enabled. The feature


## Pandas Split Data into Train / Test Datasets

In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_test, y_train, y_test = train_test_split(
    df_in_fixed, 
    df_in_fixed['label'].values, 
    test_size=0.3,
    random_state=1337,
)

len(df_train.index), len(df_test.index), y_train.shape, y_test.shape

## PySpark Split Data into Train / Test Datasets

In [ ]:
# Now for PySpark
df_train, df_test = entity_df.randomSplit([0.7, 0.3], seed=1337)

## Define the Labels for Language Extraction

In [106]:
POSITIVE = 1
NEGATIVE = 0
ABSTAIN = -1

## Setup our spaCy Preprocessors

In [107]:
import re
import jsonlines, sys
from snorkel.labeling import labeling_function, LabelingFunction
from snorkel.preprocess import preprocessor
from snorkel.preprocess.nlp import SpacyPreprocessor

spacy = SpacyPreprocessor(
    text_field='body',
    doc_field='spacy',
    memoize=True,
    language='en_core_web_lg',
    disable=['vectors'],
)

@preprocessor(memoize=True, pre=[spacy])
def restore_entity(x):
    
    entity = None
    for ent in x['spacy'].ents:
        if  ent.start == row['entity_start'] \
        and ent.end   == row['entity_end']:
            entity = ent

    if entity is None:
        raise Exception('Missing entity!')

    x['entity'] = entity
    return x

## Heuristic Functions

These Labeling Functiuons are rules based on clues that come from inspecting the data through exploratory data analysis.

In [153]:
starts_rx = re.compile('^\W')
          
@labeling_function()
def lf_starts_with_char(x):
    """NEGATIVE if starts with a non-alpha-numeric value"""
    return NEGATIVE if starts_rx.match(x['entity_text']) else ABSTAIN


number_end_rx = re.compile('^[a-zA-Z]+[0-9\W]+$')

@labeling_function()
def lf_ends_with_symbol_or_number(x):
    """POSITIVE if starts with letter and ends in number"""
    return POSITIVE if number_end_rx.match(x['entity_text']) else ABSTAIN

@labeling_function()
def lf_wrong_entity_type(x):
    return NEGATIVE if x['ent_type'] in ['PERSON', 'NORP', 'FAC', 'GPE', 'LOC', 
                                         'LAW', 'DATE', 'TIME', 'PERCENT',
                                         'MONEY', 'QUANTITY', 'ORDINAL', 'CARDINAL',] else ABSTAIN

@labeling_function()
def lf_token_count_2(x):
    """NEGATIVE if entity has more than 2 words in it"""
    return NEGATIVE if len(x['entity_text'].split(' ')) > 2 else ABSTAIN

@labeling_function()
def lf_token_count_1(x):
    """NEGATIVE if entity has more than 1 word in it"""
    return NEGATIVE if len(x['entity_text'].split(' ')) > 1 else ABSTAIN

## Keyword Lookups

These are the simplest type of Labeling Functions that use the presence of one or more words in a document to classify it as `POSITIVE` or `NEGATIVE` or to `ABSTAIN` if it is not sure.

In [147]:
#
# Make keyword LF generation
#
def keyword_lookup(x, keywords, field, label):
    """Perform lowercase matching for keyword LFs"""
    match = any(word.lower() in x[field].lower() for word in keywords)
    if match:
        return label
    return ABSTAIN

def make_keyword_lf(keywords, field='body', label=ABSTAIN, name=None):
    """Given keywords, a field to match against and a label to return, return an keyword LF"""
    if not name:
        name = f"keyword_{keywords}"
    return LabelingFunction(
        name=name,
        f=keyword_lookup,
        resources=dict(keywords=keywords, field=field, label=label),
    )

# Define keyword LFs
lf_language_keyword = make_keyword_lf(['language'], 'right_text', label=POSITIVE)
lf_written_keyword = make_keyword_lf(['written'], 'left_text', label=POSITIVE)
lf_framework_keyword = make_keyword_lf(['framework', 'package'], 'right_text', label=NEGATIVE)

#
# Use regular expressions to negate browsers
#
prefixes = ['internet', 'ie', 'firefox', 'google', 'chrome', 'apple', 'safari', 'webkit', 'gecko', 
            'opera', 'netscape', 'chromium', ]
browser_rx = re.compile(''.join(['^(?:', '|'.join(prefixes), ')']))

@labeling_function()
def lf_not_browser(x):
    """Eliminate browser false positives"""
    e = x['entity_text'].lower()
    return NEGATIVE if browser_rx.match(e) else ABSTAIN

## Distant Supervision

In these Labeling Functiuns, we use clues from external sources of data such as WikiData to classify documents. For example, we use WikiData's list of programming languages to determine whether a token is a programming language. Likewise we use the presence of part of an operating system from WikiData's name to classify documents as `NEGATIVE`, since operating systems are not progrmaming languages :)

In [110]:
from os import path
import boto3

def install(x):
    
    s3 = boto3.client('s3')

    if os.path.isfile('/tmp/programming_languages.jsonl'):
        pass
    else:
        s3.download_file('stackoverflow-events', 'programming_languages.jsonl', '/tmp/programming_languages.jsonl')

    if os.path.isfile('/tmp/operating_systems.jsonl'):
        pass
    else:
        s3.download_file('stackoverflow-events', 'operating_systems.jsonl', '/tmp/operating_systems.jsonl')

    return [1]

In [112]:
#
# Label functions using distant supervision from SPARQL/WikiData for programming languages
#
languages, lower_languages = None, None
with jsonlines.open('../data/programming_languages.jsonl', mode='r') as reader:
    languages = [x['name'] for x in reader]
    lower_languages = [x.lower() for x in languages]

@labeling_function(resources=dict(languages=languages))
def lf_matches_wikidata_langs(x, languages):
    """POSITIVE if the entity_text matches any language in list"""
    return POSITIVE if x['entity_text'] in languages else ABSTAIN

@labeling_function(resources=dict(lower_languages=lower_languages))
def lf_lower_matches_wikidata_langs(x, lower_languages):
    """POSITIVE if the lowercase entity_text matches any lowercase language in list"""
    return POSITIVE if x['entity_text'].lower() in lower_languages else ABSTAIN

# Label functions using distant supervision from SPARQL/WikiData for operating systems
oses, os_parts = [], []
with jsonlines.open('../data/operating_systems.jsonl', mode='r') as reader:
    oses = [x['name'].lower() for x in reader]
    for os in oses:
        for os_part in os.split():
            os_parts.append(os_part)

@labeling_function(resources=dict(oses=oses))
def lf_matches_wikidata_os(x, oses):
    """NEGATIVE if the lowercase entity_text matches any lowercase OS in the list"""
    return NEGATIVE if x['entity_text'].lower() in oses else ABSTAIN

@labeling_function(resources=dict(os_parts=os_parts))
def lf_matches_wikidata_os_parts(x, os_parts):
    """NEGATIVE if the lowercase entity_text matches any lowercase OS fragment in the list"""
    return NEGATIVE if x['entity_text'].lower() in os_parts else ABSTAIN

## spaCy Pattern Matching Labeling Functions

In [113]:
from spacy.matcher import Matcher
matcher = Matcher(nlp.vocab)
pattern = [{'POS': 'VERB'}, {'POS': 'ADP'}, {'POS': 'PROPN'}]
matcher.add("VERB_ADP_PROPN", None, pattern)

@labeling_function(pre=[spacy, restore_entity])
def lf_verb_in_noun(x):
    """Return positive if the pattern"""
    sp = x['spacy']
    matches = matcher(sp)
    
    found = False
    for match_id, start, end in matches:
        if end == x['entity_end']:
            pass
        if start == x['start'] - 2:            
            if sp[start].text in ['work', 'written', 'wrote']:                
                if sp[start + 1].text in ['in']:
                    return POSITIVE
    else:
        return ABSTAIN

## External Model Labeling Functions

In [10]:
import textdistance




## Define What LFs will Run

In [154]:
heuristic_lfs = [
    lf_starts_with_char,
    lf_ends_with_symbol_or_number,
    lf_wrong_entity_type,
    lf_token_count_1,
    lf_token_count_2,
]

keyword_lfs = [
    lf_language_keyword,
    lf_written_keyword,
    lf_framework_keyword,
    lf_not_browser,
]

distant_lfs = [
    lf_matches_wikidata_langs,
    lf_lower_matches_wikidata_langs,
    lf_matches_wikidata_os,
    lf_matches_wikidata_os_parts,
]

# pattern_match_lfs = [
#     lf_verb_in_noun,
# ]

lfs = heuristic_lfs + keyword_lfs + distant_lfs # + pattern_match_lfs

## Test LF Accuracy on Gold Labeled Data

We need to see how well the LFs work in classifying data with known labels, our gold labeled data.

In [155]:
from snorkel.labeling import LFAnalysis, PandasLFApplier

applier = PandasLFApplier(lfs)
L_dev = applier.apply(df_gold)
y_dev = df_gold.label.values
LFAnalysis(L_dev, lfs).lf_summary(y_dev)

/home/rjurney/anaconda3/envs/weak/lib/python3.7/site-packages/tqdm/std.py:654: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|██████████| 1290/1290 [00:00<00:00, 6542.65it/s]


,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
lf_starts_with_char,0,[0],0.009302,0.009302,0.000000,12,0,1.000000
lf_ends_with_symbol_or_number,1,[1],0.059690,0.059690,0.023256,38,39,0.493506
lf_starts_char_ends_non_char,2,[1],0.059690,0.059690,0.023256,38,39,0.493506
lf_wrong_entity_type,3,[0],0.417829,0.179070,0.030233,504,35,0.935065
lf_token_count_1,4,[0],0.154264,0.111628,0.016279,193,6,0.969849
lf_token_count_2,5,[0],0.058140,0.058140,0.001550,75,0,1.000000
keyword_['language'],6,[1],0.003876,0.001550,0.000000,3,2,0.600000
keyword_['written'],7,[1],0.002326,0.001550,0.001550,0,3,0.000000
"keyword_['framework', 'package']",8,[0],0.006202,0.003876,0.001550,6,2,0.750000
lf_not_browser,9,[0],0.011628,0.009302,0.000000,15,0,1.000000


In [ ]:
from snorkel.labeling.apply.spark import SparkLFApplier

# Get dicts that we can work with in LFs
entity_rdd = entity_df.rdd.repartition(16 * 40).map(lambda x: x.asDict())

spark_applier = SparkLFApplier(lfs)
L_train = spark_applier.apply(entity_rdd)

LFAnalysis(L_train, lfs).lf_summary()

## Analyze and Improve the Performance of our LFs

In [ ]:
(L_train != ABSTAIN).mean(axis=0)

In [ ]:
from snorkel.analysis import get_label_buckets

buckets = get_label_buckets(y_dev, L_dev[:, 1])

df_gold.iloc[buckets[NEGATIVE, POSITIVE]]

## Fit our `LabelModel`

In [ ]:
from snorkel.labeling import LabelModel

label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train, None, n_epochs=5000, log_freq=500, seed=1337)

label_model

In [ ]:
from snorkel.analysis import metric_score
from snorkel.utils import probs_to_preds

probs_dev = label_model.predict_proba(L_dev)
preds_dev = probs_to_preds(probs_dev)
print(
    f"Label model accuracy score: {metric_score(y_dev, preds_dev, probs=probs_dev, metric='accuracy')}"
)
print(
    f"Label model precision score: {metric_score(y_dev, preds_dev, probs=probs_dev, metric='precision')}"
)
print(
    f"Label model recall score: {metric_score(y_dev, preds_dev, probs=probs_dev, metric='recall')}"
)
print(
    f"Label model f1 score: {metric_score(y_dev, preds_dev, probs=probs_dev, metric='f1')}"
)
print(
    f"Label model roc-auc: {metric_score(y_dev, preds_dev, probs=probs_dev, metric='roc_auc')}"
)

## Filter out Unlabeled Data

In [ ]:
from snorkel.labeling import filter_unlabeled_dataframe

probs_train = label_model.predict_proba(L_train)
df_train_filtered, probs_train_filtered = filter_unlabeled_dataframe(
    X=df_train, y=probs_train, L=L_train
)

df_train_filtered.head(3)

## Build our Discriminative Bidirectional LSTM Model

In [ ]:
# Note: this model is pulled from the Snorkel Spouse Example: https://www.snorkel.org/use-cases/spouse-demo

from typing import Tuple
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.layers import (
    Bidirectional,
    Concatenate,
    Dense,
    Embedding,
    Input,
    LSTM,
)


# elmo = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)


def get_feature_arrays(df: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray]:
    """Get np arrays of upto max_length tokens and person idxs."""
    left = df['left_tokens_text']
    right = df['right_tokens_text']

    def pad_or_truncate(l, max_length=40):
        pad_length = max_length - len(l)
        padding = [""] * pad_length
        left_values = l[:max_length]
        padded_values = np.append(left_values, padding)
        return padded_values

    left_tokens = np.array(list(map(pad_or_truncate, left)))
    right_tokens = np.array(list(map(pad_or_truncate, right)))
    return left_tokens, right_tokens


def bilstm(
    tokens: tf.Tensor,
    rnn_state_size: int = 64,
    num_buckets: int = 40000,
    embed_dim: int = 36,
):
    ids = tf.strings.to_hash_bucket(tokens, num_buckets)
    embedded_input = Embedding(num_buckets, embed_dim)(ids)
    return Bidirectional(LSTM(rnn_state_size, activation=tf.nn.relu))(
        embedded_input, mask=tf.strings.length(tokens)
    )


# def char_emb(
#     tokens: tf.Tensor,
# ):
#     embeddings = elmo(
#     ["the cat is on the mat", "dogs are in the fog"],
#     signature="default",
#     as_dict=True)["elmo"]


def get_model(
    rnn_state_size: int = 64, num_buckets: int = 40000, embed_dim: int = 12
) -> tf.keras.Model:
    """
    Return LSTM model for predicting label probabilities.
    Args:
        rnn_state_size: LSTM state size.
        num_buckets: Number of buckets to hash strings to integers.
        embed_dim: Size of token embeddings.
    Returns:
        model: A compiled LSTM model.
    """
    left_ph = Input((None,), dtype="string")
    # entity_ph = Input((None,), dtype="string")
    right_ph = Input((None,), dtype="string")
    left_embs = bilstm(left_ph, rnn_state_size, num_buckets, embed_dim)
    # char_embs = char_emb()
    right_embs = bilstm(right_ph, rnn_state_size, num_buckets, embed_dim)
    layer = Concatenate(1)([left_embs, right_embs])
    layer = Dense(64, activation=tf.nn.relu)(layer)
    layer = Dense(32, activation=tf.nn.relu)(layer)
    probabilities = Dense(2, activation=tf.nn.softmax)(layer)
    model = tf.keras.Model(inputs=[left_ph, right_ph], outputs=probabilities)
    model.compile(tf.compat.v1.train.AdagradOptimizer(0.1), "categorical_crossentropy")
    return model

In [ ]:
X_train = get_feature_arrays(df_train_filtered)
model = get_model()
batch_size = 64
model.fit(X_train, probs_train_filtered, batch_size=batch_size, epochs=30)

## Evalute Discriminative Model

In [ ]:
X_gold = get_feature_arrays(df_gold)
probs_gold = model.predict(X_gold)
preds_gold = probs_to_preds(probs_gold)

print(
    f"Gold accuracy score                       : {metric_score(y_dev, preds=preds_gold, probs=probs_gold, metric='accuracy')}"
)
print(
    f"Gold precision score                      : {metric_score(y_dev, preds=preds_gold, probs=probs_gold, metric='precision')}"
)
print(
    f"Gold recall score                         : {metric_score(y_dev, preds=preds_gold, probs=probs_gold, metric='recall')}"
)
print(
    f"Gold F1 when trained with hard labels     : {metric_score(y_dev, preds=preds_gold, metric='f1')}"
)
print(
    f"Gold ROC-AUC when trained with soft labels: {metric_score(y_dev, probs=probs_gold, metric='roc_auc')}"
)

In [ ]:
df_gold['pred_prob'] = probs_gold[:,1]
df_gold['pred_label'] = preds_gold
df_gold.head(5)

In [ ]:
# get_feature_arrays(df_test.head(3))
df_test['left_tokens_text'].iloc[0]
df_test.head(3)

# Improving the Model

## Exploratory Data Analysis to Drive Keyword Labeling Functions



In [137]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier


left_vectorizer = TfidfVectorizer(
    stop_words='english',
    min_df=3,
    norm=None,
    
)
left_text_vec = left_vectorizer.fit_transform(
    df_gold['left_text'].apply(lambda x: x.lower())
)
labels = df_gold['label']

left_clf = RandomForestClassifier(n_estimators=500)
left_clf.fit(left_text_vec, labels)

list(reversed(sorted(zip(left_vectorizer.get_feature_names(), left_clf.feature_importances_), key=lambda x: x[1])))

[('using', 0.027076161652506533),
 ('learn', 0.02043907554690311),
 ('scala', 0.019284526142435466),
 ('got', 0.018911144814287788),
 ('like', 0.018433102655341253),
 ('code', 0.016536506587976812),
 ('server', 0.015973379397164165),
 ('consider', 0.01459127352046222),
 ('restrict', 0.013775668645336366),
 ('ve', 0.013436828528643061),
 ('want', 0.013248763375124295),
 ('file', 0.013170904670913476),
 ('warnings', 0.013154068135351492),
 ('sql', 0.012814544040309423),
 ('text', 0.012778470024587657),
 ('question', 0.012177488317227198),
 ('platform', 0.012002576629853358),
 ('findbugs', 0.011937173185628423),
 ('use', 0.011699183915944914),
 ('developers', 0.011519910807053127),
 ('create', 0.011286571409254457),
 ('string', 0.011157315599050322),
 ('easily', 0.01110462264498605),
 ('custom', 0.010896963441336385),
 ('equivalent', 0.010766846238537891),
 ('work', 0.010019847591914361),
 ('passing', 0.00996182012355815),
 ('just', 0.009896730633612055),
 ('comment', 0.009795753101247497

In [138]:
import lime
from sklearn.metrics import f1_score

class_names = ['other', 'programming language']

pred = left_clf.predict(left_text_vec)
f1_score(labels, pred, average='binary')

from lime import lime_text
from sklearn.pipeline import make_pipeline
c = make_pipeline(vectorizer, left_clf)

from lime.lime_text import LimeTextExplainer
explainer = LimeTextExplainer(class_names=class_names)

for idx in range(5, 15):
    exp = explainer.explain_instance(df_gold['left_text'][idx], c.predict_proba, num_features=6)
    print('Left text: %s' % df_gold['left_text'][idx].lower())
    print('Entity text: %s' % df_gold['entity_text'][idx])
    print('Probability(programming language) =', c.predict_proba([df_gold['left_text'][idx]])[0,1])
    print('True class: %s' % class_names[labels[idx]])
    print(exp.as_list())

Left text: fwiw, this is a stock
Entity text: Python
Probability(programming language) = 0.14928891092957172
True class: programming language
[('stock', 2.730762452581165e-32), ('FWIW', 1.610594647763694e-32), ('this', -1.2729093318557785e-32), ('is', 1.2491703875691557e-32), ('a', 9.951486353532807e-33)]
Left text: iterate through the keys of a
Entity text: Perl
Probability(programming language) = 0.14928891092957172
True class: programming language
[('the', 1.5323136856856308e-32), ('keys', 1.4652129091281027e-32), ('a', 1.211716601413053e-32), ('iterate', 4.658608836777033e-33), ('through', 3.3635166326249394e-33), ('of', 2.7481120108781645e-33)]
Left text: hash? if i have a
Entity text: Perl
Probability(programming language) = 0.14928891092957172
True class: programming language
[('hash', 0.0), ('If', 0.0), ('I', 0.0), ('have', 0.0), ('a', 0.0)]
Left text: , and is one of the
Entity text: two
Probability(programming language) = 0.14928891092957172
True class: other
[('is', 2.073306

ValueError: low >= high

In [139]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier


right_vectorizer = CountVectorizer(
    stop_words='english',
#     min_df=3,
#     norm=None,
    
)
right_text_vec = right_vectorizer.fit_transform(
    df_gold['right_text'].apply(lambda x: x.lower())
)
labels = df_gold['label']

right_clf = RandomForestClassifier(n_estimators=500)
right_clf.fit(right_text_vec, labels)

list(reversed(sorted(zip(right_vectorizer.get_feature_names(), right_clf.feature_importances_), key=lambda x: x[1])))

[('handle', 0.013343591723685615),
 ('equivalent', 0.012943545997779396),
 ('casing', 0.012539697261859295),
 ('functional', 0.012315094298080715),
 ('script', 0.012256117562778964),
 ('perl', 0.0119842962313325),
 ('string', 0.010563042159591215),
 ('c11', 0.009354070985061332),
 ('continue', 0.009147551688293527),
 ('file', 0.00873333959218845),
 ('csv', 0.008611310687897276),
 ('developers', 0.007365866295801967),
 ('create', 0.007291473824677684),
 ('python', 0.007288992648213816),
 ('language', 0.007070163492785972),
 ('doing', 0.006982939386126604),
 ('haskell', 0.006886807541236523),
 ('harder', 0.006813510481818956),
 ('question', 0.006775385317550928),
 ('seen', 0.006711397998472881),
 ('forth', 0.006618303395414207),
 ('jquery', 0.006542004791008239),
 ('thanks', 0.006537122288198064),
 ('judging', 0.006504243267559236),
 ('looking', 0.006496848386979279),
 ('table', 0.006484797129140765),
 ('learning', 0.006482357939019178),
 ('speak', 0.006450205459930621),
 ('installation'

In [140]:
import lime
from sklearn.metrics import f1_score

class_names = ['other', 'programming language']

pred = clf.predict(right_text_vec)
f1_score(labels, pred, average='binary')

from lime import lime_text
from sklearn.pipeline import make_pipeline
c = make_pipeline(vectorizer, clf)

from lime.lime_text import LimeTextExplainer
explainer = LimeTextExplainer(class_names=class_names)

for idx in range(0, 10):
    exp = explainer.explain_instance(df_gold['right_text'][idx], c.predict_proba, num_features=6)
    print('Right  text: %s' % df_gold['right_text'][idx].lower())
    print('Entity text: %s' % df_gold['entity_text'][idx])
    print('Probability(programming language) =', c.predict_proba([df_gold['right_text'][idx]])[0,1])
    print('True class:  %s' % class_names[labels[idx]])
    print(exp.as_list())

ValueError: Number of features of the model must match the input. Model n_features is 1123 and input n_features is 289 

In [149]:
lf_developer_keyword = make_keyword_lf(['developer', 'developers'], 'right_text', label=NEGATIVE)
lf_left_using_keyword = make_keyword_lf(['using'], 'left_text', label=NEGATIVE, name='using_left')
lf_right_using_keyword = make_keyword_lf(['using'], 'right_text', label=NEGATIVE, name='using_right')

additional_lfs = [
    lf_developer_keyword,
    lf_left_using_keyword,
    lf_right_using_keyword,
]

new_lfs = lfs + additional_lfs

In [151]:
from snorkel.labeling import LFAnalysis, PandasLFApplier

applier = PandasLFApplier(new_lfs)
L_dev = applier.apply(df_gold)
y_dev = df_gold.label.values
LFAnalysis(L_dev, new_lfs).lf_summary(y_dev)

/home/rjurney/anaconda3/envs/weak/lib/python3.7/site-packages/tqdm/std.py:654: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|██████████| 1290/1290 [00:00<00:00, 6281.83it/s]


,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
lf_starts_with_char,0,[0],0.009302,0.009302,0.000000,12,0,1.000000
lf_ends_with_symbol_or_number,1,[1],0.059690,0.039535,0.027132,38,39,0.493506
lf_wrong_entity_type,2,[0],0.417829,0.193023,0.030233,504,35,0.935065
lf_token_count_1,3,[0],0.154264,0.115504,0.016279,193,6,0.969849
lf_token_count_2,4,[0],0.058140,0.058140,0.001550,75,0,1.000000
keyword_['language'],5,[1],0.003876,0.001550,0.000000,3,2,0.600000
keyword_['written'],6,[1],0.002326,0.001550,0.001550,0,3,0.000000
"keyword_['framework', 'package']",7,[0],0.006202,0.003876,0.001550,6,2,0.750000
lf_not_browser,8,[0],0.011628,0.009302,0.000000,15,0,1.000000
lf_matches_wikidata_langs,9,[1],0.079070,0.079070,0.027132,92,10,0.901961
